# How to spinfoam amplitudes
## Contracting the EPRL vertex amplitudes

In this notebook we contract the *previously computed* EPRL vertices in order to obtain the Δ4 amplitude.

Since the required vertices have already been computed, 
the cells below can be compiled just installing the required packages, even if the sl2cfoam-next is not installed in your system.

Obviously, if you try to assemble vertices which have not been pre-computed in this repository you get an error. 

<img src="Pics/Delta_4.SVG" alt="Drawing" style="width: 700px;"/>

### Building the Δ4 amplitude

In [35]:
# required pkgs  
using HalfIntegers
using JLD2   
using DelimitedFiles
using DataFrames

In [36]:
# Barbero-Immirzi parameter
γ = 1.2

# current folder
root_dir = pwd()

# homogeneous cut-off 
Δl = 15;

Let's start by showing a way to contract amplitudes.

Notice that **in julia best performance is achieved by using functions**.

Although performance is not the priority of this notebook, for those interested in writing beautiful and efficient julia code we refer to [julia performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/).

In [6]:
# value of all boundary spins (symmetric configuration)    
j = 1.0

# value of all boundary intertwiners (symmetric configuration)   
i_boundary = 0

# julia arrays start at index 1   
i_b = i_boundary + 1    
    
# folder with stored vertex amplitudes 
vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                     
                   
Δ4_amp = 0.0 
    
# if all boundary spins are integers and equal to j, bulk spin has range [0, 3j]   
j_bulk_min, j_bulk_max = 0, 3j 
    
for j_bulk = j_bulk_min:j_bulk_max

fulltensor_to_load = "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2"
if(!isfile(fulltensor_to_load)) error("vertex fulltensor for γ=$(γ) j=$(j) Δl=$(Δl) not found, it must be computed first") end   
@load "$(fulltensor_to_load)" vertex 
                    
# ranges of intertwiner spaces touched by the bulk spin      
D = size(vertex[i_b,:,:,i_b,i_b])[1] # also [2] has the same range          
                    
Δ4_partial_amp = 0.0
  
for i_1234 in 1:D, i_1235 in 1:D, i_1236 in 1:D, i_1237 in 1:D
@inbounds Δ4_partial_amp += vertex[i_b,i_1234,i_1235,i_b,i_b]*vertex[i_b,i_1235,i_1236,i_b,i_b]*
                            vertex[i_b,i_1236,i_1237,i_b,i_b]*vertex[i_b,i_1237,i_1234,i_b,i_b]     
end     

# dimension of bulk spin     
Δ4_partial_amp *= (2j_bulk + 1)
            
Δ4_amp += Δ4_partial_amp  
                    
end # j_bulk cycle
               
@show Δ4_amp

Δ4_amp = 1.849656117029305e-37


1.849656117029305e-37

For convenience, we write a function that contracts all the required vertices for given lists of boundary parameters in the symmetric configuration.

We collect all the amplitudes in a matrix for increasing values of the cut-off Δl and different intertwiners configurations, 
then we plot and store the amplitudes. 

Notice that julia plots below are just a useful tool for qualitative analysis, as they are not optimal.

In [38]:
# required pkgs
using Plots
using Plots.PlotMeasures
using LaTeXStrings

# set backend and style
pgfplotsx()
theme(:default)
default(markerstrokewidth = 0, 
        markerstrokealpha = 0, 
        linewidth = 2, 
        #grid=:none,   
        opacity = 0.8,
        gridlinewidth= 0.5,
        markersize = 10,
        markershape = :circle,
        tickfontsize = 18, 
        size = (900, 600), 
        yformatter = string,
        legendfontsize = 24, 
        guidefontsize=20,
        titlefontsize = 37,
        legend=(0.03,0.98), 
        foreground_color_axis = "black",
        foreground_color_border = "black",
        foreground_color_guide = "darkorange",
        foreground_color_text = "black",
        guidefontcolor = "black", 
        plot_titlefontcolor = "black",
        titlefontcolor = "black", 
        shape=[:circle])

Plots.PGFPlotsXBackend()

Plots.PGFPlotsXBackend()

In [81]:
function Δ4_assemble(Immirzi_list, j_boundary_list, Δl_max)
    
for γ in Immirzi_list

   for j in j_boundary_list    
           
      i_boundary_min, i_boundary_max = 0, convert(Int, 2j)
        
      # matrix in which the computed amplitudes will be stored         
      Δ4_amplitudes = zeros(Δl_max + 1, i_boundary_max + 1)           
      column_labels = String[] 
      ampls_to_plot = Vector{Float64}[]      

      for i_boundary = i_boundary_min:i_boundary_max
          
         i_b = i_boundary + 1 
         push!(column_labels, "i=$(i_boundary)")        
    
         for Δl = 0:Δl_max

            vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                           
            Δ4_amp = 0.0 
                
            if (iszero(j%1)) j_bulk_min = 0 else j_bulk_min = 0.5 end       
            j_bulk_max = 3j
    
            for j_bulk = j_bulk_min:j_bulk_max

            fulltensor_to_load = "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2"
            if(!isfile(fulltensor_to_load)) error("vertex fulltensor for γ=$(γ) j=$(j) Δl=$(Δl) not found, it must be computed first") end    
            @load "$(fulltensor_to_load)" vertex 
                        
               D = size(vertex[i_b,:,:,i_b,i_b])[1]                              
               Δ4_partial_amp = 0.0
  
                  for i_1234 in 1:D, i_1235 in 1:D, i_1236 in 1:D, i_1237 in 1:D
                  @inbounds Δ4_partial_amp += vertex[i_b,i_1234,i_1235,i_b,i_b]*vertex[i_b,i_1235,i_1236,i_b,i_b]*
                                              vertex[i_b,i_1236,i_1237,i_b,i_b]*vertex[i_b,i_1237,i_1234,i_b,i_b]     
                  end     
   
                  Δ4_partial_amp *= (2j_bulk + 1)            
                  Δ4_amp += Δ4_partial_amp 
                        
                  end # j_bulk cycle
                
               Δ4_amplitudes[Δl+1,i_b] = Δ4_amp       
        
               # storing the Δ4 amplitude in JLD2 format    
               Δ4_JLD2_path = "$(root_dir)/Delta_4_ampls/Immirzi_$(γ)/j_$(j)/JLD2_format/i_$(i_boundary)/"
               mkpath(Δ4_JLD2_path)         
               @save "$(Δ4_JLD2_path)/Dl_$(Δl).jld2" Δ4_amp
        
               end # cut-off cycle    
                
            push!(ampls_to_plot, Δ4_amplitudes[:,i_b])
        
            end # i_boundary cycle    
         
         # plotting the amplitudes   
         # TODO: add dynamic labels to intertwiners   
         plot(0:Δl_max, 
         ampls_to_plot, 
         xticks = 0:Δl_max,       
         markersize = 10,
         markershape = :circle,
         palette = :RdYlGn_9,
         ylims = (0, maximum(Δ4_amplitudes)),    
         label="",       
         legend=(0.04,0.95))    
         xlabel!(L" \Delta l" )
         ylabel!(L" A_{\Delta 4}" )
         title!(L"\mathrm{\Delta 4\ amplitude\ \left( j=%$(j)\ \gamma=%$(γ) \right) }\ ")
         savefig("$(root_dir)/Plots/Delta_4_j=$(j)_gamma=$(γ).pdf")     
                  
         # storing the Δ4 amplitudes in CSV format
         Δ4_CSV_path = "$(root_dir)/Delta_4_ampls/Immirzi_$(γ)/j_$(j)/CSV_format" 
         mkpath(Δ4_CSV_path)
         open("$(Δ4_CSV_path)/Delta_4_ampls_Dl_max_$(Δl_max).csv", "w") do io
         writedlm(io, Δ4_amplitudes)
         end   
          
         # notice that i-th row corresponds to Δl=i-1 
         Amplitudes = DataFrame(Δ4_amplitudes, column_labels)    
         println("Δ4 amplitudes for j = $(j) and γ = $(γ) are:\n", Amplitudes, "\n\n")  

   end # j_boundary cycle       

end # Immirzi cycle   
    
end # end of function

Δ4_assemble (generic function with 1 method)

In [82]:
# list of Immirzi parameters 
Immirzi_list = [0.1, 1.0, 1.2]

# list of boundary spins (symmetric configurations)
j_boundary_list = [1.0]

# maximal cut-off  
Δl_max = 25;

In [83]:
Δ4_assemble(Immirzi_list, j_boundary_list, Δl_max)

Δ4 amplitudes for j = 1.0 and γ = 0.1 are:
26×3 DataFrame
 Row │ i=0          i=1          i=2         
     │ Float64      Float64      Float64     
─────┼───────────────────────────────────────
   1 │ 8.11368e-27  5.95039e-26  1.73228e-25
   2 │ 3.87335e-26  2.76091e-25  8.4097e-25
   3 │ 7.74983e-26  5.21764e-25  1.64612e-24
   4 │ 1.11081e-25  7.12266e-25  2.30166e-24
   5 │ 1.37469e-25  8.48548e-25  2.78686e-24
   6 │ 1.57478e-25  9.44201e-25  3.13607e-24
   7 │ 1.72678e-25  1.01247e-24  3.38984e-24
   8 │ 1.84306e-25  1.06208e-24  3.57681e-24
   9 │ 1.93335e-25  1.099e-24    3.71739e-24
  10 │ 2.00436e-25  1.12701e-24  3.82497e-24
  11 │ 2.06105e-25  1.1487e-24   3.90883e-24
  12 │ 2.10686e-25  1.16578e-24  3.97521e-24
  13 │ 2.14436e-25  1.17945e-24  4.02856e-24
  14 │ 2.17538e-25  1.19053e-24  4.07199e-24
  15 │ 2.20131e-25  1.19964e-24  4.10777e-24
  16 │ 2.22318e-25  1.20721e-24  4.13757e-24
  17 │ 2.24178e-25  1.21355e-24  4.16263e-24
  18 │ 2.25773e-25  1.21893e-24  4.18389